In [1]:
words = open("names.txt", "r").read().splitlines()
words[0]

'emma'

In [2]:
xin = []
yin = []
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        # print(ch1, ch2, ch3)
        xin.append(ch1)
        xin.append(ch2)
        yin.append(ch3)
        # if ch3 == '.':
        #     break
print(words[:3])
print(xin[:20])
print(yin[:10])

['emma', 'olivia', 'ava']
['.', 'e', 'e', 'm', 'm', 'm', 'm', 'a', '.', 'o', 'o', 'l', 'l', 'i', 'i', 'v', 'v', 'i', 'i', 'a']
['m', 'm', 'a', '.', 'l', 'i', 'v', 'i', 'a', '.']


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

def stoi2(s, t):
    return stoi[s] * len(stoi) + stoi[t]

def itos2(i):
    return itos[i // len(stoi)] + itos[i % len(stoi)]

print(stoi['m'])
print(stoi2('e', 'e'))
print(itos2(15))
print(itos[13])

13
140
.o
m


In [4]:
import torch

In [5]:
xs, ys = [], []

for ch1, ch2, ch3 in zip(xin[::2], xin[1::2], yin):
    xs.append(stoi2(ch1, ch2))
    ys.append(stoi[ch3])

# print(xin[:10], yin[:5])
# print(xs[:5], ys[:5])


In [9]:
print(xs.shape)
print(ys.shape)


torch.Size([196113])
torch.Size([196113])


In [7]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=729).float()
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

In [ ]:
char_counts = {char: (ys == index).sum().item() for char, index in stoi.items()}

print("Character counts in ys:")
for char, count in char_counts.items():
    print(f"{char}: {count}")

total_chars = sum(char_counts.values())
print(f"\nTotal characters: {total_chars}")
print(f"Total elements in ys: {len(ys)}")

In [45]:
learning_rate = 0.1  # Reduced learning rate
optimizer = torch.optim.Adam([W], lr=learning_rate)

for k in range(1000):  # Increased number of iterations
    optimizer.zero_grad()
    logits = xenc @ W
    loss = F.cross_entropy(logits, ys)
    loss.backward()
    optimizer.step()
    
    if k % 100 == 0:
        print(f'Iteration {k}, Loss: {loss.item()}')

Iteration 0, Loss: 3.72312331199646
Iteration 100, Loss: 2.0675392150878906
Iteration 200, Loss: 2.0641331672668457
Iteration 300, Loss: 2.063145160675049
Iteration 400, Loss: 2.0627074241638184
Iteration 500, Loss: 2.062469244003296
Iteration 600, Loss: 2.0623230934143066
Iteration 700, Loss: 2.062225818634033
Iteration 800, Loss: 2.06215763092041
Iteration 900, Loss: 2.0621070861816406


In [48]:
def get_probability_distribution(input_chars):
    # Convert input characters to index
    ix = stoi2(input_chars[0], input_chars[1])
    
    # Create one-hot encoding for the input
    xenc1 = F.one_hot(torch.tensor([ix]), num_classes=729).float()
    
    # Calculate logits
    logits1 = xenc1 @ W
    
    # Apply softmax to get probabilities
    probs = F.softmax(logits1, dim=1)
    
    # Convert to numpy for easier handling
    probs_np = probs.detach().numpy()[0]
    
    # Create a dictionary of character probabilities
    char_probs = {itos[i]: prob for i, prob in enumerate(probs_np)}
    
    return char_probs

# Example usage
input_chars = 'ia'  # Example input
prob_dist = get_probability_distribution(input_chars)

# Print probabilities sorted by value
for char, prob in sorted(prob_dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{char}: {prob:.4f}")

.: 0.3693
n: 0.3231
h: 0.1885
s: 0.0225
r: 0.0221
m: 0.0213
l: 0.0102
a: 0.0057
g: 0.0057
v: 0.0049
i: 0.0041
d: 0.0033
y: 0.0033
t: 0.0029
b: 0.0025
u: 0.0020
k: 0.0020
z: 0.0012
p: 0.0008
c: 0.0008
e: 0.0008
q: 0.0008
o: 0.0008
j: 0.0008
f: 0.0004
x: 0.0002
w: 0.0000


In [ ]:
def generate_names(num_names):
    g = torch.Generator().manual_seed(2147483648)
    names = []
    
    for _ in range(num_names):
        name = []
        ch1, ch2 = '.', '.'  # Start with '..' as the initial bigram
        
        while True:
            prob_dist = get_probability_distribution(ch1 + ch2)
            probs = torch.tensor(list(prob_dist.values()))
            next_char_idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
            next_char = itos[next_char_idx]
            
            if next_char == '.':  # End of name
                break
            
            name.append(next_char)
            ch1, ch2 = ch2, next_char
        
        names.append(''.join(name))
    
    return names

# Generate and print 5 names
generated_names = generate_names(15)
for i, name in enumerate(generated_names, 1):
    print(f"Name {i}: {name}")

Name 1: oan
Name 2: phion
Name 3: slea
Name 4: gann
Name 5: treiaghalah
Name 6: iz
Name 7: zir
Name 8: uzana
Name 9: zinsham
Name 10: zyra
Name 11: yah
Name 12: olla
Name 13: karydne
Name 14: dael
Name 15: gue
